**Title** mainERLATFEED

**Description**: This is the primary script for Hannah Clayton's analysis of gray whale feeding events. It serves as a place to run all the necessary sub functions such as audits and flukebeat detection. Some components require going to MATLAB and returning with inputs.

**Author:** Hannah Clayton

**About:** 

**Requites:**

**Notes:**
- 01.25.2024. HMC initialized script.
- 03.01.2024. HMC. Polishing code for public git repo for BLS2024.

**To-do:**
- Change importing final feed to only input if it exists, otherwise it throw

**Known Bugs***
- Plotting of BORIS observations sometimes do not seem to line up exactly. Only seems to be an issue in deployments where videos were fragmented and there was potentially an offset that needs to be addressed.

In [1]:
# -- LOAD LIBRARIES -- #
import scipy.io as scio
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import requests
import openpyxl
from datetime import datetime as dt
from datetime import timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import mplcursors
import os
from itertools import chain
from matplotlib.widgets import Button
from pathlib import Path
import xlrd
import warnings
import matplotlib.dates as mdates
import seaborn as sns
import random
import csv
import scipy
import math as m
from ipywidgets import interact, widgets, Layout, Button
import random

%run fxBORIS.ipynb                   
%run getfeedBORISObs.ipynb
%run makePRHObs.ipynb
%run plotBORISObsDepthRoll.ipynb
%run plotQCRolls.ipynb
%run plotQCSlim.ipynb
%run getTagOnOff.ipynb
%run getTides.ipynb
%run plotTideRollDepth.ipynb
%run descTidalChar.ipynb

warnings.simplefilter('ignore')


In [2]:
# -- LOAD DATA -- #
# ------------------------ User defined variables --------------------------------------- #
fs = 10                                                                                         # What's the sampling rate of the prh/speed file you want to pull? (in Hz)
os.chdir('C:\\01_Work\\1Project Home\\1Research\\GRAYS\\_ERLATFEED')                            # path to master project folder
depID = input('Which deployment is being analyzed?')                                            # get the deployment ID for naming and pulling files
# --------------------------------------------------------------------------------------- #
datpth = "C:\\01_Work\\1Project Home\\1Research\\GRAYS\\_tag data\\"                            # path to tag data with by-deployment structures
cdir = str(os.getcwd())                                                                         # your current drive 
outloc = str(cdir + '\\analysisoutput\\' + depID + "\\")                                        # path to output files

# CREATE A OUTPUT DIR IF IT DOESN'T EXIST
if os.path.exists(outloc) == False:                                             
    os.mkdir(outloc)
    os.mkdir(outloc+"\\rejected rolls\\")
else:
    print('Output directory already exists...')

prh = scio.loadmat(datpth + depID + "\\" + depID + " 10Hzprh.mat")                              # prh file
#speed = scio.loadmat(datpth + depID + "\\" + depID + "_10HzSpeed.mat")                         # speed file
#speed = pd.DataFrame(speed['spd'])                                                             # fix importing from matlab
feed = pd.read_csv(datpth + depID + "\\feed\\"+ depID + "_rolls.csv")                           # file exported from matlab with all 'possible' feeding events 
dems = pd.read_excel("C:\\01_Work\\1Project Home\\1Research\\GRAYS\\mastertracker_GRAYS.xlsx")  # demographic information
mastfeed = pd.read_excel(cdir + '\\analysisoutput\\master-feed+tide.xlsx')                      # excel file with all feeding data from all deployments 
mastsum = pd.read_excel(cdir + '\\analysisoutput\\master-sumfeed.xlsx')                         # master summary data

#
finfeed = str(cdir + '\\analysisoutput\\'+depID+'\\'+depID+'-feed+tide.xlsx')
if os.path.isfile(finfeed) == True:
    finalfeed = pd.read_excel(cdir + '\\analysisoutput\\'+depID+'\\'+depID+'-feed+tide.xlsx')   # TODO! if you've already processed you can read in the final feeding structure
else:
    print('No final feed file.')

doesexist = prh.get('vidDN')
# IF THERE IS NO VIDEO DATA FOR DPLYMNT, SKIP
if doesexist is None or len(doesexist) == 0:
    nxtcell = 0
    print('No video data to process, continue to cell 2.')
    novid = 1
# IF THERE IS BORIS AUDIT FILES, LET'S PROCESS THEM
else:
    if os.path.exists(str(datpth + depID+'\\BORIS\\') == True):
        nxtcell = 1
        novid = 0
        print('Video data to process, continue to cell 1.')
    else: 
        print('Deployment has video, but no BORIS files found.')
        novid = 1
        nxtcell = 0
        rolls = feed
        rolls['ObsInd'] = None

Which deployment is being analyzed? er230416-70


Output directory already exists...
Video data to process, continue to cell 1.


In [4]:
from datetime import datetime, timedelta
# CELL1. This takes roll data from matlab kinematic audit, observations from BORIS video audits, and prh video data to link behavioral 
# observations to potential feeding events (roll) and make a prh-indexed data structure for plotting observational data
if nxtcell > 0:
    # GET BORIS DATA, COMPILE AND CLEAN IT, AND SAVE THE COMPILED DATA 
    boris,viddat = mutateBORISandVID(depID,outloc,prh)

    # GET OBSERVATIONS OCCURRING DURING ROLLS AND ADD THOSE TO ROLL DF
    rolls = getfeedBORISObs(feed, boris, outloc, depID)
    prhObs, indTagOn, tagoffInd, tagoffDS, tagonDS, periodsNOCAM = makePRHObs(boris, prh)
    plotBORIS(prhObs, outloc, depID)

    nxtcell = 1
    print('BORIS data added to roll data and prhObs df made, continue to cell 3.')
    
else:
    nxtcell = 1
    rolls = feed
    rolls['ObsInd'] = None
    print('No BORIS data to compile, continue to cell 2.')
    

No BORIS data to compile, continue to cell 2.


In [5]:
%matplotlib inline

#CELL2. This generates a plot of roll, triaxial accelerometer data, depth, 
# and any video observations during the roll of interest for final expert 
# confirmation of feeding

# checks for rejected roll file to see if this step has already been completed
if not os.path.exists(datpth + depID + "\\feed\\"+ depID + "_rejectedrolls.xlsx"):
    toRemove = []
    row_selector = widgets.IntSlider(value=0, min=0, max=(len(rolls))-1, step=1, description='Row Number',
                                     style={'description_width': 'initial'}, layout=Layout(width='100%'))
    interact(plotQCRolls, rownum=row_selector, prh=widgets.fixed(prh), rolls=widgets.fixed(rolls))
else:
    rejfeed = pd.read_excel(datpth + depID + "\\feed\\"+ depID + "_rejectedrolls.xlsx")
    toRemove = rejfeed['indRolls'].tolist()
    print('Loaded rejected rolls and feed data structures...')



Loaded rejected rolls and feed data structures...


In [8]:
toRemove = list(set(toRemove)) # get rid of potential duplicates
feed = rolls.drop(toRemove)
#feed = feed[feed['toRemove'] != 1]
feed['indRolls'] = feed.index
feed = feed.reset_index(drop=True)
rejfeed = rolls.iloc[toRemove]
rejfeed['indRolls'] = rejfeed.index
rejfeed = rejfeed.reset_index(drop=True)
rejfeed.to_excel(datpth + depID + "\\feed\\"+ depID + "_rejectedrolls.xlsx")
print("Saved discarded rolls to excel in data directory.")

if not any(os.scandir(outloc+"rejected rolls\\")):
    for i in range(len(rejfeed)):
        plotQCSlim(rejfeed,i,prh,outloc,depID)
else:
    print('Plots already generated for rejected rolls...')

Saved discarded rolls to excel in data directory.
Plots already generated for rejected rolls...


In [9]:
%matplotlib qt

feed['sFEDS'] = pd.to_datetime(feed['sFEDS']).dt.strftime('%Y-%m-%d %H:%M:%S')
# now lets prep the feed events data and get it ready for the signal audit
# GRAB DEMOGRAPHIC INFO                           
demINFO= dems[dems['ID'] == str(depID)]                             # subset to only this whale's data
feed['name'] = str(demINFO['CRCID'].values[0])
feed['sex'] = str(demINFO['SEX'].values[0])
feed['length_m'] = demINFO['UASMEASURE_m'].values[0]
feed['yrArrived'] = demINFO['FIRSTSEEN-YYYY'].values[0]
feed['numCohort'] = demINFO['COHORT'].values[0]
feed['nDeployments'] = demINFO['n_DPLYMNTS'].values[0]
feed['durDeploy'] = demINFO['sum_TAGON-hhmmss'].values[0]

print('Demographic info added to feed structure...')

# ADD FEEDING INFO
feed['n'] = range(1, len(feed)+1)
feed['nFeedEvents'] = feed['n'].max()                             # total number of feeding events
# TIME DATA
feed['durFP_s'] = (feed['eFPI'] - feed['sFPI'])/fs                # duration spent in feeding position
feed['durEFP_s'] = (feed['sFPI'] - feed['sFEI'])/fs               # time it took to enter feeding position
feed['durLFP_s'] = (feed['eFEI'] - feed['eFPI'])/fs 
feed['hrFeed'] = np.nan
feed['sFEDS'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S').hour) # buggy
#groupcountsHrFeed = feed.groupby('hrFeed').size()

feed = feed.drop(32)
feed = feed.reset_index(drop=True)

for i in range(len(feed)):
    if np.isnan(feed.loc[i, 'sFPI']) == True or np.isnan(feed.loc[i, 'eFPI']) == True:
        print(i)
        sI =  feed.loc[i, 'sFEI']  # Calculate mean, stdev in roll,pitch in feeding position
        eI =  feed.loc[i, 'eFEI']
        rolldat = pd.DataFrame(prh['roll'][sI:eI + 1]) 
        fig, ax = plt.subplots()
        ax.plot(rolldat, label='Roll (rad)',  linewidth=2) 
        selpoints = plt.ginput(2, show_clicks=True,mouse_add=1, mouse_pop=2, mouse_stop=3)  # generate cursor
        plt.draw()
        feed.loc[i, 'sFPI'] = round(selpoints[0][0]+sI)
        feed.loc[i, 'eFPI'] = round(selpoints[1][0]+sI)
        feed.loc[i, 'durFP_s'] = (feed.loc[i, 'eFPI']-feed.loc[i, 'sFPI'])/fs
        feed.loc[i, 'durLFP_s'] = (feed.loc[i, 'eFEI']-feed.loc[i, 'eFPI'])/fs
        feed.loc[i, 'durEFP_s'] = (feed.loc[i, 'sFPI']-feed.loc[i, 'sFEI'])/fs

    else:
        continue

for i in range(len(feed)):
    sI = feed.loc[i, 'sFPI'].astype(int)
    eI = feed.loc[i, 'eFPI'].astype(int)
    rolldat = prh['roll'][sI:eI + 1]
    #print(rolldat)
    pitchdat = prh['pitch'][sI:eI + 1]
    feed.loc[i, 'meanRollFP'] = m.degrees(np.mean(rolldat))
    feed.loc[i,'stdevRollFP'] = m.degrees(np.std(rolldat))
    feed.loc[i,'rangeRollFP'] = m.degrees(np.ptp(rolldat))
    feed.loc[i,'meanPitchFP'] = np.mean(pitchdat)
    feed.loc[i,'stdevPitchFP'] = np.std(pitchdat)
    feed.loc[i,'rangePitchFP'] = np.ptp(pitchdat)
    
feed

Demographic info added to feed structure...
9
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


,sFEI,eFEI,durFE_s,sFEDN,sFEDS,camon,flagQAQC,comment,toRemove,sFPI,...,durFP_s,durEFP_s,durLFP_s,hrFeed,meanRollFP,stdevRollFP,rangeRollFP,meanPitchFP,stdevPitchFP,rangePitchFP
0,73686,74280,59.4,738992.487141,2023-04-16 11:41:29,0,0.0,NaN,0,73717.0,...,50.1,3.1,6.2,NaN,110.654629,3.777885,23.826820,-0.434011,0.045291,0.275390
1,74657,75309,65.2,738992.488267,2023-04-16 11:43:06,0,0.0,NaN,0,74699.0,...,53.1,4.2,7.9,NaN,114.719673,4.649335,28.783026,-0.427966,0.035983,0.266576
2,78193,78554,36.1,738992.492356,2023-04-16 11:48:59,0,0.0,NaN,0,78227.0,...,27.2,3.4,5.5,NaN,108.652794,3.885746,21.744900,-0.408616,0.039832,0.325664
3,81112,81597,48.5,738992.495735,2023-04-16 11:53:51,0,0.0,NaN,0,81148.0,...,39.1,3.6,5.8,NaN,113.524559,6.346760,26.291615,-0.426360,0.037434,0.327270
4,82278,82723,44.5,738992.497089,2023-04-16 11:55:48,0,0.0,NaN,0,82309.0,...,35.1,3.1,6.3,NaN,113.655157,3.852154,20.875323,-0.432050,0.040227,0.347012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,306198,306355,15.7,738992.756247,2023-04-16 18:08:59,0,NaN,NaN,1,306229.0,...,7.3,3.1,5.3,NaN,109.995901,4.614116,19.794665,-0.460083,0.041262,0.220707
202,307119,307508,38.9,738992.757313,2023-04-16 18:10:31,0,NaN,NaN,1,307144.0,...,30.6,2.5,5.8,NaN,119.125067,6.017203,28.236989,-0.478118,0.031480,0.203645
203,307600,308134,53.4,738992.757869,2023-04-16 18:11:19,0,NaN,NaN,1,307633.0,...,44.2,3.3,5.9,NaN,117.819287,4.946987,24.064166,-0.466737,0.040414,0.283251
204,308280,308754,47.4,738992.758656,2023-04-16 18:12:27,0,NaN,NaN,1,308323.0,...,36.9,4.3,6.2,NaN,116.586774,4.238997,29.055339,-0.466809,0.040846,0.314517


In [ ]:
#print(feed.iloc[118])


In [10]:
# CELL#. Perform tidal analysis. This block loads in tidal data from csv in the tag data directory, creates tidal periods and quarters 
# based on whether the tag was on and there could be or is feeding, and further separates the tidal period into quarters.
# These quarters are creates by finding the center time between a high and a low tide and creating two quarters.
# the tidal height and decription of tide characteristics is then coded into feeding data and returned.
#1. Create or load tidal data
# CHECK IF DS VAR EXISTS
if not 'DS' in prh:
    def matlab_datenum_to_datetime(matlab_datenum):
        python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum % 1) - timedelta(days=366)
        return python_datetime
    prh['DN'] = pd.Series(prh.get('DN').squeeze())
    prh['DS'] = prh['DN'].apply(matlab_datenum_to_datetime)
    indTagOn, tagonDS, indTagOff, tagoffDS = getTagOnOff(prh)
else:
    print('prh datestring variable already exists...')
prh['DS_HHMMSS'] = pd.to_datetime(prh['DS'].dt.strftime('%Y-%m-%d %H:%M:%S'))

# GET TIDAL DATA
tides, hilotides, depTides = getTides(prh, feed, datpth, tagonDS, tagoffDS)
#hilotides, depTides, feed = descTidalChar(hilotides, prh, feed, depTides, tagonDS, tagoffDS)

#feed

Loading tidal data...


In [11]:


def isinTimeRange(searchdf, starttime, endtime):
    # Convert strings to datetime objects
    starttime = pd.to_datetime(starttime)
    endtime = pd.to_datetime(endtime)
    # Check if the given time is within the range
    return ((searchdf >= starttime) & (searchdf <= endtime)).any()


def makeTagon(hilotides,i,prh):
    searhDT = hilotides.loc[i, 'Date Time']
    boolTagOn = any(prh['DS_HHMMSS'].isin([searhDT])) # is the tag on?
    if boolTagOn == True: 
        hilotides.loc[i,'tagon'] = bool(1) 
    else: 
        hilotides.loc[i,'tagon'] = bool(0)

    return(hilotides)


def setStartandEndPeriod(hilotides, i, tagonDS, tagoffDS):
        periodstart = pd.to_datetime(hilotides.loc[i, 'Date Time'])
        periodend = pd.to_datetime(hilotides.loc[i+1, 'Date Time'])
        hilotides.loc[i,'periodStartDS'] = periodstart
        hilotides.loc[i,'periodStopDS'] = periodend
        hilotides.loc[i, 'boolFeed'] = isinTimeRange(feed['sFEDS'], periodstart, periodend)
        #hilotides.loc[i, 'centerPeriod'] = (periodstart+(periodend-periodstart))/2
        timediff1 = periodend-periodstart
        hilotides.loc[i, 'centerPeriod'] = periodstart+timediff1/2
        
        # if the tag was put on after the start of the period and before the end
        if tagonDS > hilotides.loc[i,'periodStartDS'] and tagonDS < hilotides.loc[i,'periodStopDS']:
            periodstart = tagonDS
        else:
            print("")
            
        # if the tag came off after the start of the period and before the end
        if tagoffDS > hilotides.loc[i,'periodStartDS'] and tagoffDS < hilotides.loc[i,'periodStopDS']:
            periodend = tagoffDS
        else:
            print('')
            
        # FIND THE CENTER TIME TO MAKE YOUR QUARTERS
        timediff2 = periodend-periodstart
        hilotides.loc[i, 'durPeriod'] = timediff2
        
        # if the tag came off before the center of the period, find the duration and set the second quarter to nan
        if periodend >= hilotides.loc[i, 'centerPeriod']:
            hilotides.loc[i, 'durQ1'] = hilotides.loc[i, 'centerPeriod'] - hilotides.loc[i,'periodStartDS'] 
            hilotides.loc[i, 'durQ2'] = periodend - hilotides.loc[i, 'centerPeriod'] 
            hilotides.loc[i, 'durQ1'] = hilotides.loc[i, 'durPeriod']
            #hilotides.loc[i, 'durQ1'] = periodend-periodstart 
        # if the tag came off after the center period, 
        else: 
            hilotides.loc[i, 'durQ1'] = hilotides.loc[i, 'durPeriod']
            hilotides.loc[i, 'durQ2'] =  pd.Timedelta(0)
        return(hilotides)


# SET AS DT VARS
feed['sFEDS'] = pd.to_datetime(feed['sFEDS'])
hilotides['Date Time'] = pd.to_datetime(hilotides['Date Time'])
depTides['DateTime'] = pd.to_datetime(depTides['DateTime'])
tagonDS = tagonDS.strftime('%Y-%m-%d %H:%M:%S')
tagoffDS = tagoffDS.strftime('%Y-%m-%d %H:%M:%S')
tagonDS = pd.to_datetime(tagonDS)
tagoffDS = pd.to_datetime(tagoffDS)

for i in range(len(hilotides)):
    hilotides = makeTagon(hilotides,i,prh)
    if i < len(hilotides)-1:
        if hilotides.loc[i, ' Type'] == 'L' and hilotides.loc[i+1, ' Type'] == 'H':
            hilotides.loc[i, 'risefall'] = 'R'    
        else:
            hilotides.loc[i, 'risefall'] = 'F'
        hilotides = setStartandEndPeriod(hilotides, i, tagonDS, tagoffDS)
    else: 

        hilotides.loc[i,'periodStartDS'] = hilotides.loc[i, 'Date Time']
        hilotides.loc[i,'periodStopDS'] =  np.nan
        hilotides.loc[i, 'boolFeed'] = False
        hilotides.loc[i, 'risefall'] = np.nan
        hilotides.loc[i, 'centerPeriod'] = np.nan
        hilotides.loc[i, 'durPeriod'] =  pd.Timedelta(0)
        hilotides.loc[i, 'durQ1'] =  pd.Timedelta(0)
        hilotides.loc[i, 'durQ2'] =  pd.Timedelta(0)     

hilotides = hilotides[~((hilotides['tagon'] == False) & (hilotides['tagon'].shift(-1) == False))].reset_index(drop = True)

hilotides['numTPeriod'] = (hilotides.index)
if hilotides.loc[0, 'numTPeriod'] == 0:
    hilotides['numTPeriod'] = (hilotides.index)+1
else:
    hilotides['numTPeriod'] = (hilotides.index)

for i in range(len(hilotides)-1):
    if hilotides.loc[i, 'durQ1'] == hilotides.loc[i, 'durPeriod']:
        hilotides.loc[i, 'durQ1'] = hilotides.loc[i, 'durPeriod']- hilotides.loc[i, 'durQ2']
    else:
        print('')
hilotides.to_excel(outloc+depID+"-hilotides.xlsx")        

hilotides

,Date Time,Prediction,Type,tagon,risefall,periodStartDS,periodStopDS,boolFeed,centerPeriod,durPeriod,durQ1,durQ2,numTPeriod
0,2023-04-16 09:17:00,4.568,L,False,R,2023-04-16 09:17:00,2023-04-16 14:39:00,True,2023-04-16 11:58:00,0 days 04:45:40,0 days 02:04:40,0 days 02:41:00,1
1,2023-04-16 14:39:00,8.646,H,True,F,2023-04-16 14:39:00,2023-04-16 20:57:00,True,2023-04-16 17:48:00,0 days 06:18:00,0 days 03:09:00,0 days 03:09:00,2
2,2023-04-16 20:57:00,1.163,L,True,R,2023-04-16 20:57:00,2023-04-17 03:49:00,False,2023-04-17 00:23:00,0 days 03:30:45,0 days 03:26:00,0 days 00:04:45,3
3,2023-04-18 22:42:00,2.780,L,False,NaN,2023-04-18 22:42:00,NaT,False,NaT,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,4


In [ ]:
hilotides.loc[0, 'durQ1'] = pd.Timedelta(0)
hilotides.loc[0, 'durQ2'] = hilotides.loc[0, 'durPeriod']
hilotides

In [12]:



for i in range(len(depTides)):
    currentTime = pd.to_datetime(depTides.loc[i, 'DateTime'])
    diff = currentTime - hilotides['Date Time']
    indHiLoTides = diff[diff > pd.Timedelta(0)].idxmin()
    depTides.loc[i, 'numTPeriod'] = hilotides.loc[indHiLoTides, 'numTPeriod']
    depTides.loc[i, 'durPeriod'] = hilotides.loc[indHiLoTides, 'durPeriod']

    depTides.loc[i, 'risefall'] = hilotides.loc[indHiLoTides, 'risefall']
    centerPeriod = hilotides.loc[indHiLoTides, 'centerPeriod']
    thisPeriod = str(int(depTides.loc[i, 'numTPeriod']))
    
    if currentTime <= centerPeriod:
        #thisPeriod = str(int(depTides.loc[i, 'numTPeriod']))
        depTides.loc[i, 'numTQuarter'] = str(1)
        depTides.loc[i, 'durQuarter'] = hilotides.loc[indHiLoTides, 'durQ1']
        depTides.loc[i, 'descTide'] = f"P{thisPeriod}{depTides.loc[i, 'risefall']}Q{depTides.loc[i, 'numTQuarter']}"    
    else:
        depTides.loc[i, 'numTQuarter'] = str(2)
        depTides.loc[i, 'durQuarter'] = hilotides.loc[indHiLoTides, 'durQ2']
        depTides.loc[i, 'descTide'] = f"P{thisPeriod}{depTides.loc[i, 'risefall']}Q{depTides.loc[i, 'numTQuarter']}"  

for i in range(len(feed)):
# get the tidal height for this feeding event and the tidal decription
    currentTime = pd.to_datetime(feed.loc[i, 'sFEDS'])
    inddepTides = (depTides['DateTime'] - currentTime).abs().idxmin()
    closestTime = depTides.loc[inddepTides, 'DateTime']
    feed.loc[i, 'tidalheight_ft'] = depTides.loc[inddepTides, 'Prediction']
    feed.loc[i, 'descTide'] = depTides.loc[inddepTides, 'descTide']
    feed.loc[i, 'risefall'] = depTides.loc[inddepTides, 'risefall']

    feed.loc[i, 'numTPeriod'] = depTides.loc[inddepTides, 'numTPeriod']
    feed.loc[i, 'durPeriod'] = depTides.loc[inddepTides, 'durPeriod']
    feed.loc[i, 'durQuarter'] = depTides.loc[inddepTides, 'durQuarter']

#plotTideRollDepth(depTides, prh, tagonDS, tagoffDS, outloc, depID, hilotides)

d = scipy.zeros(len(depTides['Prediction']))
commonxlim = (tagonDS, tagoffDS)

fig, (ax, ax2) = plt.subplots(2,1,figsize = (24,8))
ax.plot(depTides['DateTime'], depTides['Prediction'], color = 'black', linestyle = 'dashed', linewidth = 4)
ax.fill_between(depTides['DateTime'], depTides['Prediction'], where=depTides['Prediction']>=d, interpolate=True, color='whitesmoke')
ax.set_xlim(commonxlim)
ax.set(ylim=[0, max(depTides['Prediction'])+1])
ax.set_xlabel('')
ax.set_xticks([])
ax.set_ylabel('Tidal Height (ft)')
ax.vlines(x=hilotides['centerPeriod'], ymin=0, ymax=max(depTides['Prediction'])+1, color = 'black',linewidth = 3)
ax.vlines(x=hilotides['periodStartDS'], ymin=0, ymax=max(depTides['Prediction'])+1, color='black',linewidth = 3)


ax2.plot(prh['DS'], prh['roll'], color = 'black')
ax2.set_xlim(commonxlim)
ax2.set_xlabel('')
ax2.set_xticks([])
ax2.set_ylabel('Roll (rad)')




plt.show()

In [13]:
# now lets get the densities and counts of feeding events in each group
for i in range(len(feed)):
    feed.loc[i, 'depID'] = depID
    curDescTide = feed.loc[i, 'descTide']
    grpCount = feed[feed['descTide'] == curDescTide].shape[0]
    feed.loc[i, 'descTideCount'] = grpCount
    decHrs = feed.loc[i, 'durQuarter'].total_seconds()/3600 # convert the quarter duration to number of hours
    feed.loc[i, 'descTideDensityHr'] = grpCount/decHrs

feed.to_excel(outloc+depID+"-feed+tide.xlsx")
mdat = pd.concat([mastfeed, feed], ignore_index = True)
mdat.to_excel(cdir + '\\analysisoutput\\master-feed+tide.xlsx')

# Now lets create a summary df for this deployment
# I want:
# mean and stdev of durFP_s
# mean,stdev of mean Roll
# mean, stdev of stdev of roll in feeding position
# mean, stdev of range in roll in feeding position
# mean, stdev of mean pitch in feeding position
# mean, stdev of normalized period feeding rate
sumDep = pd.DataFrame()
sumDep.loc[1,'depID'] = depID
sumDep.loc[1,'whaleName'] = str(demINFO['CRCID'].values[0])
sumDep.loc[1, 'meanDurFP_s'] = feed['durFP_s'].mean()
sumDep.loc[1, 'stdDurFP_s'] = feed['durFP_s'].std()
sumDep.loc[1, 'meanMeanRollFP'] = feed['meanRollFP'].mean()
sumDep.loc[1, 'stdMeanRollFP'] = feed['meanRollFP'].std()
sumDep.loc[1, 'meanStdRollFP'] = feed['stdevRollFP'].mean()
sumDep.loc[1, 'stdStdRollFP'] = feed['stdevRollFP'].std()
sumDep.loc[1, 'meanRangeRollFP'] = feed['rangeRollFP'].mean()
sumDep.loc[1, 'stdRangeRollFP'] = feed['rangeRollFP'].std()
sumDep.loc[1, 'meanMeanPitchRollFP'] = feed['meanPitchFP'].mean()
sumDep.loc[1, 'stdMeanPitchFP'] = feed['meanPitchFP'].std()
sumDep.loc[1, 'meanRangePitchFP'] = feed['rangePitchFP'].mean()
sumDep.loc[1, 'stdRangePitchFP'] = feed['rangePitchFP'].std()
sumDep.loc[1, 'meanQFeedRate'] = feed['descTideDensityHr'].mean()
sumDep.loc[1, 'stdQFeedRate'] = feed['descTideDensityHr'].std()
sumDep.to_excel(outloc+depID+'-sumfeed.xlsx')

sdat = pd.concat([mastsum, sumDep], ignore_index = True)

sdat.to_excel(cdir + '\\analysisoutput\\master-sumfeed.xlsx')
